3. Сгенерировать последовательности, которые состоят из цифр (от 0 до 9) и задаются следующим образом:
x - последовательность цифр
y1 = x1 
yi = xi + x1
Если yi >= 10 то yi = yi - 10
Научить модель рекуррентной нейронной сети предсказывать yi по xi Использовать: RNN, LSTM, GRU


In [2]:
import numpy as np
import torch
from itertools import permutations
import random
import time
import torch.nn.functional as F

Подготовка данных
Функция для перестановки заданного списка
Генерируем последовательность из цифр от 0-9. Пример:

[1, 2, 3, 4, 6, 8, 0, 7, 9, 5]
[1, 2, 3, 4, 6, 8, 0, 9, 5, 7]
[1, 2, 3, 4, 6, 8, 0, 9, 7, 5]
[1, 2, 3, 4, 6, 9, 5, 7, 8, 0]
[1, 2, 3, 4, 6, 9, 5, 7, 0, 8]
[1, 2, 3, 4, 6, 9, 5, 8, 7, 0]
[1, 2, 3, 4, 6, 9, 5, 8, 0, 7]

In [5]:
# р закоментировано, так как размер файла становится слишком больши
def permutation(lst):
 
    # Если lst пуст, то перестановок нет
    if len(lst) == 0:
        return []
 
    # если 1 элемент, то одна перестановка возможна
   
    if len(lst) == 1:
        return [lst]
 
    # Найдите перестановки для lst, если они существую
    # более чем 1 символ
 
    l = [] # пустой список, в котором будет храниться текущая перестановка
 
    # Повторите ввод (lst) и вычислите перестановку
    for i in range(len(lst)):
       m = lst[i]
 
       # Extract lst[i] or m from the list.  remLst is
       # remaining list
       remLst = lst[:i] + lst[i+1:]
 
       # Генерация всех перестановок, где m является первым элементом
       
       for p in permutation(remLst):
           l.append([m] + p)
    return l
 
 
# тест функции, добавим все перестановки в список
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
list = []
for p in permutation(data):
    list.append(p)
    #print (p)

In [6]:
list[:5]

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
 [1, 2, 3, 4, 5, 6, 7, 8, 0, 9],
 [1, 2, 3, 4, 5, 6, 7, 9, 8, 0],
 [1, 2, 3, 4, 5, 6, 7, 9, 0, 8],
 [1, 2, 3, 4, 5, 6, 7, 0, 8, 9]]

In [7]:
#длина полученного списка
len(list)

3628800

In [8]:
#отбираем из списка по индексу кратный 50, так как list слишком большой
Xdata = []
for m in range(0, len(list)):
    if m %50==0:
        Xdata.append(list[m])

In [9]:
len(Xdata)

72576

In [10]:
Xdata[:5]

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
 [1, 2, 3, 4, 5, 8, 6, 9, 7, 0],
 [1, 2, 3, 4, 5, 0, 6, 9, 7, 8],
 [1, 2, 3, 4, 6, 7, 8, 5, 9, 0],
 [1, 2, 3, 4, 6, 9, 7, 8, 5, 0]]

In [11]:
#функция - прописываем условия для Y
def create_dataset(df):
    set = []
    ydata = []
    set.append(df[0])
    for i in range(1, len(df)):
        set.append(df[i]+df[0])
    for j in range(0, len(set)):
        if set[j] >= 10:
            ydata.append(set[j]-10)
        else:
            ydata.append(set[j]) 
    return ydata

In [12]:
#тест
df = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
create_dataset(df)

[1, 3, 4, 5, 6, 7, 8, 9, 0, 1]

In [13]:
#применяем фугкцию для всех элементов списка
Ydata=[]
for k in Xdata:
    Ydata.append(create_dataset(k))

In [14]:
len(Ydata)

72576

In [15]:
Ydata[:5]

[[1, 3, 4, 5, 6, 7, 8, 9, 0, 1],
 [1, 3, 4, 5, 6, 9, 7, 0, 8, 1],
 [1, 3, 4, 5, 6, 1, 7, 0, 8, 9],
 [1, 3, 4, 5, 7, 8, 9, 6, 0, 1],
 [1, 3, 4, 5, 7, 0, 8, 9, 6, 1]]

In [16]:
# делим на train/test
num_trn = int(0.75*len(Xdata))                
X_trn, Y_trn = Xdata[:num_trn], Ydata[:num_trn]
X_test, Y_test = Xdata[num_trn:], Ydata[num_trn:]

In [17]:
len(X_trn)

54432

In [18]:
len(Y_trn)

54432

In [19]:
len(X_test)

18144

In [20]:
len(Y_test)

18144

RNN

In [22]:
class RNNModel(torch.nn.Module):
    
    def __init__(self):
        super(RNNModel, self).__init__()
        self.rnn = torch.nn.RNN(input_size=10, hidden_size=100, batch_first=True)
        self.linear = torch.nn.Linear(100, 10)

    def forward(self, X, state=None):
        #o -выход
        #h - последнее скрытое состояние
        o, h = self.rnn(X)
        out = self.linear(o)
        return out

Применялась MSELoss, а не CrossEntropyLoss
Так как  по типу тензоров X, Y при обучении были ошибки и не расчитывался loss, если тип int
X_trn = torch.tensor(X_trn, dtype = torch.float32).view(len(X_trn), 1, -1)
Y_trn = torch.tensor(Y_trn, dtype = torch.float32).view(len(Y_trn), -1)

In [24]:
model = RNNModel()
optimizer =torch.optim.Adam(model.parameters())
loss = torch.nn.MSELoss()
#loss = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

Обучаем модель

In [26]:
for epoch in range(50):
    train_loss, train_acc, iter_num = .0, .0, .0
    start_epoch_time = time.time()
    model.train()
    X_trn = torch.tensor(X_trn, dtype = torch.float32).view(len(X_trn), 1, -1)
    Y_trn = torch.tensor(Y_trn, dtype = torch.float32).view(len(Y_trn), -1)
    #print(X_trn.shape)
    #print(Y_trn.shape)
    optimizer.zero_grad() 
    out = model.forward(X_trn) 
    #print(out.shape)
    out = out.view(len(Y_trn), -1)
    #print(out.shape)
    #print(Y_trn.shape)
    l = loss(out, Y_trn)
    train_loss += l.item()
    l.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, loss: {train_loss:.4f}",end=" | ")

Epoch: 0, loss: 28.8521 | Epoch: 1, loss: 28.0552 | Epoch: 2, loss: 27.2745 | 

C:\Users\user\AppData\Local\Temp\ipykernel_3632\2169003629.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_trn = torch.tensor(X_trn, dtype = torch.float32).view(len(X_trn), 1, -1)
C:\Users\user\AppData\Local\Temp\ipykernel_3632\2169003629.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_trn = torch.tensor(Y_trn, dtype = torch.float32).view(len(Y_trn), -1)


Epoch: 3, loss: 26.5106 | Epoch: 4, loss: 25.7638 | Epoch: 5, loss: 25.0338 | Epoch: 6, loss: 24.3198 | Epoch: 7, loss: 23.6207 | Epoch: 8, loss: 22.9353 | Epoch: 9, loss: 22.2626 | Epoch: 10, loss: 21.6016 | Epoch: 11, loss: 20.9519 | Epoch: 12, loss: 20.3131 | Epoch: 13, loss: 19.6851 | Epoch: 14, loss: 19.0682 | Epoch: 15, loss: 18.4627 | Epoch: 16, loss: 17.8689 | Epoch: 17, loss: 17.2875 | Epoch: 18, loss: 16.7193 | Epoch: 19, loss: 16.1649 | Epoch: 20, loss: 15.6253 | Epoch: 21, loss: 15.1013 | Epoch: 22, loss: 14.5938 | Epoch: 23, loss: 14.1038 | Epoch: 24, loss: 13.6321 | Epoch: 25, loss: 13.1796 | Epoch: 26, loss: 12.7469 | Epoch: 27, loss: 12.3347 | Epoch: 28, loss: 11.9435 | Epoch: 29, loss: 11.5736 | Epoch: 30, loss: 11.2252 | Epoch: 31, loss: 10.8986 | Epoch: 32, loss: 10.5936 | Epoch: 33, loss: 10.3100 | Epoch: 34, loss: 10.0476 | Epoch: 35, loss: 9.8059 | Epoch: 36, loss: 9.5844 | Epoch: 37, loss: 9.3824 | Epoch: 38, loss: 9.1992 | Epoch: 39, loss: 9.0340 | Epoch: 40, lo

Проверка на тестовый данных

In [28]:
for epoch in range(50):
    test_loss = .0
    start_epoch_time = time.time()
    model.eval()
    X_test = torch.tensor(X_test, dtype = torch.float32).view(len(X_test), 1, -1)
    Y_test = torch.tensor(Y_test, dtype = torch.float32).view(len(Y_test), -1)
    out = model.forward(X_test) 
    out = out.view(len(Y_test), -1)
    l = loss(out, Y_test)
    test_loss += l.item()
    l.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, loss: {test_loss:.4f}",end=" | ")

Epoch: 0, loss: 9.2253 | Epoch: 1, loss: 9.1901 | Epoch: 2, loss: 9.1422 | Epoch: 3, loss: 9.0779 | Epoch: 4, loss: 8.9977 | Epoch: 5, loss: 8.9052 | Epoch: 6, loss: 8.8052 | 

C:\Users\user\AppData\Local\Temp\ipykernel_3632\3419201947.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype = torch.float32).view(len(X_test), 1, -1)
C:\Users\user\AppData\Local\Temp\ipykernel_3632\3419201947.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_test = torch.tensor(Y_test, dtype = torch.float32).view(len(Y_test), -1)


Epoch: 7, loss: 8.7029 | Epoch: 8, loss: 8.6034 | Epoch: 9, loss: 8.5108 | Epoch: 10, loss: 8.4288 | Epoch: 11, loss: 8.3601 | Epoch: 12, loss: 8.3069 | Epoch: 13, loss: 8.2707 | Epoch: 14, loss: 8.2526 | Epoch: 15, loss: 8.2529 | Epoch: 16, loss: 8.2719 | Epoch: 17, loss: 8.3090 | Epoch: 18, loss: 8.3634 | Epoch: 19, loss: 8.4337 | Epoch: 20, loss: 8.5176 | Epoch: 21, loss: 8.6125 | Epoch: 22, loss: 8.7152 | Epoch: 23, loss: 8.8220 | Epoch: 24, loss: 8.9294 | Epoch: 25, loss: 9.0340 | Epoch: 26, loss: 9.1327 | Epoch: 27, loss: 9.2228 | Epoch: 28, loss: 9.3018 | Epoch: 29, loss: 9.3670 | Epoch: 30, loss: 9.4161 | Epoch: 31, loss: 9.4469 | Epoch: 32, loss: 9.4579 | Epoch: 33, loss: 9.4480 | Epoch: 34, loss: 9.4172 | Epoch: 35, loss: 9.3666 | Epoch: 36, loss: 9.2984 | Epoch: 37, loss: 9.2163 | Epoch: 38, loss: 9.1250 | Epoch: 39, loss: 9.0297 | Epoch: 40, loss: 8.9361 | Epoch: 41, loss: 8.8493 | Epoch: 42, loss: 8.7739 | Epoch: 43, loss: 8.7133 | Epoch: 44, loss: 8.6696 | Epoch: 45, loss

Результаты неудовлетворительные по всем моделям (RNN/LSTM/GRU). Возможно некорректные данные для обучения или ошибки в модели